# Image to Text to Audio file

In [ ]:
# !pip uninstall googletrans
# !pip install googletrans==3.1.0a0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab Notebooks/POC_Projects/Parag_Letter.jfif'

open(path,mode ='r')

import PIL
from PIL import ImageDraw
im = PIL.Image.open(path)
im

In [ ]:
!pip install easyocr
#change run type to GPU
!pip install googletrans
from googletrans import Translator

import easyocr
reader = easyocr.Reader(['en']) #IMP 'hi'
translator = Translator()
bounds = reader.readtext(path,add_margin = 0.1,width_ths=0.5, link_threshold=0.4,decoder='beamsearch', blocklist='=-' )
bounds

def draw_boxes(image,bounds,color= 'yellow',width =2):
  draw = ImageDraw.Draw(image)
  for bound in bounds:
    p0,p1,p2,p3=bound[0]
    draw.line([*p0,*p1,*p2,*p3,*p0], fill = color, width = width)
  return image

draw_boxes(im, bounds)

In [ ]:
text_list = reader.readtext(path,add_margin = 0.55,width_ths=0.7, link_threshold=0.8,decoder='beamsearch', blocklist='=-',detail = 0 )
# print(text_list)

text_comb =' '.join(text_list) #changed into a single line
# print(text_comb)
text_comb

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/POC_Projects'
fhand = open('OCR_Output.txt','w')
fhand.write(text_comb) # Erass all, already writtena and write what has been passed
fhand.close()

## Text to Audio Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/POC_Projects"
# !ls

# make a request to google to get synthesis  
# t1 = gtts.gTTS("Welcome to javaTpoint")
!pip install gTTS
import gtts 
with open('/content/drive/My Drive/Colab Notebooks/POC_Projects/OCR_Output.txt', 'r') as file:
    data = file.read().replace('~', '')#data = file.read().replace('\n', '')

fhand = data.replace("\ufeff", "")
print(fhand[0:100])
t1 = gtts.gTTS(fhand)

# save the audio file  
t1.save("welcome.mp3")
from IPython.display import Audio
t1 = gtts.gTTS(fhand,lang='en')#'hi'
Audio('welcome.mp3')
# Audio('welcome.mp3',autoplay=True)

In [ ]:
t1_hi_ascent = gtts.gTTS(fhand,lang='hi')
Audio('welcome.mp3')

In [ ]:
# print(translator.detect(fhand))

In [ ]:
# text_en = translator.translate(fhand,src ='en')
# print(text_en.text)

In [ ]:
# text_en_hi = translator.translate(fhand,src ='en',dest = 'hi')
# print(text_en_hi.text)

# PDF to Text extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/POC_Projects/Resume_Vikas Kumar_Data Scientist.pdf'

!apt-get install poppler-utils
!pip install pdf2image

from pdf2image import convert_from_path
images = convert_from_path(path)
from IPython.display import display, Image
display(images[0]) # for multiple pages, you can use images[1] etc

In [ ]:
!pip install easyocr
import numpy as np
import easyocr
reader = easyocr.Reader(['en']) #IMP

bounds = reader.readtext(np.array(images[0]),min_size = 0,slope_ths=0.2,ycenter_ths = 0.7, height_ths = 0.6, width_ths = 0.8, decoder = 'beamsearch', beamWidth = 10)
bounds

# import PIL
from PIL import ImageDraw
def draw_boxes (image,bounds, color = 'yellow', width = 2):
  draw = ImageDraw.Draw(image)
  for bound in bounds:
    p0, p1, p2, p3 = bound[0]
    draw.line([*p0,*p1,*p2,*p3,*p0],fill=color, width = width)
  return image

draw_boxes(images[0], bounds)

In [ ]:
# bounds[1][1]
text = ''
for i in range(len(bounds)):
  text = text + bounds[i][1] + '\n'

print(text)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

from spacy import displacy
displacy.render (nlp(doc.text),style = 'ent',jupyter = True)

# Video to Text Extraction

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install deepspeech==0.9.3

!pip install wget 
!pip install easyocr 
!pip install deepspeech-gpu==0.9.3 
!pip install pafy 
!pip install youtube-dl 

!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm 
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

from deepspeech import Model 
import numpy as np 
import os 
import wave 
import easyocr 
import pafy 
from IPython.display import Audio, Image 
from IPython.display import YouTubeVideo 

model_file_path = 'deepspeech-0.9.3-models.pbmm' 
lm_file_path = 'deepspeech-0.9.3-models.scorer' 
beam_width = 500  #more beam width, more accuracy but it will take more time also
lm_alpha = 0.93 
lm_beta = 1.18 
model = Model(model_file_path) 
model.enableExternalScorer(lm_file_path) 
model.setScorerAlphaBeta(lm_alpha, lm_beta) 
model.setBeamWidth(beam_width) 
YOUTUBE_ID = 'zcAbBWdCxD4' 
YouTubeVideo(YOUTUBE_ID)

URL='https://www.youtube.com/watch\?v\='+ YOUTUBE_ID 
!youtube-dl --extract-audio --audio-format wav --output "gates.%(ext)s" $URL 

!ffmpeg -i gates.wav -vn -ar 16000 -ac 1 SteveJobs.wav #16000 is the framerate


In [ ]:
Audio('SteveJobs.wav')

In [ ]:
stream = model.createStream()

def read_wav_file(filename):
  with wave.open(filename,'rb') as w:
    rate = w.getframerate()
    frames = w.getnframes()
    buffer = w.readframes(frames)
  return buffer, rate  

from IPython.display import clear_output 
def transcribe_streaming(audio_file): 
  buffer, rate = read_wav_file(audio_file) 
  offset=0 
  batch_size=65536 
  text=''


  while offset < len(buffer): 
    end_offset=offset+batch_size 
    chunk=buffer[offset:end_offset]
    datal6 = np.frombuffer(chunk, dtype=np.int16)


    stream.feedAudioContent(datal6) 
    text=stream.intermediateDecode() 
    #clear output(wait=True) 
    print(text) 
    offset=end_offset 
  return True 

# transcribe_streaming('SteveJobs.wav')

In [ ]:
def transcribe(audio_file):
  buffer, rate =read_wav_file(audio_file)
  data16 = np.frombuffer(buffer,dtype = np.int16)
  return model.sttWithMetadata(data16)

transcribe('SteveJobs.wav')


In [ ]:

import wget
thumbnailurl = 'https://img.youtube.com/vi/'+YOUTUBE_ID + '/hqdefault.jpg'
print(thumbnailurl)
thumbnail = wget.download(thumbnailurl)

In [ ]:
# !ls

# thumbnail

# Image(thumbnail)

In [ ]:
# reader = easyocr.Reader(['en'])

# bounds = reader.readtext(thumbnail,width_ths =0.7,link_threshold =0.8,detail = 0)
# print(bounds)

# video = pafy.new(URL.replace('\\',''))

# print(video.title)

# print(video)